# LIBRO - Quick Single Bug Test
Test pipeline on one GHRB bug to verify everything works

In [ ]:
#@title 1. Setup: Mount Drive and Check GPU
from google.colab import drive
import os
import torch

# Mount drive
drive.mount('/content/drive')

# Check GPU
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
#@title 2. Install Dependencies
!pip install -q transformers torch accelerate huggingface_hub sentencepiece
!pip install -q tqdm pyyaml requests

print("✓ Dependencies installed")

In [ ]:
#@title 3. Clone Repository from GitHub

# Replace with your GitHub repo URL
GITHUB_REPO = "https://github.com/YOUR_USERNAME/libro_replication.git"  #@param {type:"string"}

# Clone into Drive for persistence
project_dir = '/content/drive/MyDrive/libro_replication'

if not os.path.exists(project_dir):
    !git clone {GITHUB_REPO} {project_dir}
    print(f"✓ Cloned repo to {project_dir}")
else:
    print(f"✓ Repo already exists at {project_dir}")
    # Update repo
    !cd {project_dir} && git pull

os.chdir(project_dir)
print(f"Working directory: {os.getcwd()}")

In [ ]:
#@title 4. Download GHRB Dataset (One-time, ~10 mins)
import subprocess
from pathlib import Path

# Check if already downloaded
ghrb_tar = Path("data/GHRB/ghrb-repos.tar.gz")

if not ghrb_tar.exists():
    print("Downloading GHRB dataset (~1.5GB)...")
    !mkdir -p data/GHRB
    !cd data/GHRB && wget -O ghrb-repos.tar.gz "https://figshare.com/ndownloader/files/37005352?private_link=de40ea0a3dea94560e84"
    print("✓ Download complete")
else:
    print("✓ GHRB archive already exists")

# Extract if needed
repos_dir = Path("data/GHRB/repos")
if not repos_dir.exists() or not list(repos_dir.iterdir()):
    print("Extracting repositories...")
    !cd data/GHRB && tar -xzf ghrb-repos.tar.gz
    print("✓ Extraction complete")
else:
    print("✓ Repositories already extracted")

# Verify
repos = list(repos_dir.iterdir())
print(f"\n✓ Found {len(repos)} repositories")

In [ ]:
#@title 5. Extract Bug Reports
!python3 scripts/extract_ghrb_bugs_enhanced.py

# Verify
import json
with open('data/GHRB/bug_reports.json') as f:
    bugs = json.load(f)

print(f"\n✓ Loaded {len(bugs)} bug reports")
print(f"\nFirst bug:")
print(f"  Project: {bugs[0]['project']}")
print(f"  Issue: #{bugs[0]['issue_number']}")
print(f"  Title: {bugs[0]['title']}")

In [ ]:
#@title 6. Login to HuggingFace
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#@title 7. Test on Single Bug
import sys
sys.path.append('.')

import json
import logging
from src.libro_pipeline import LIBROPipeline

logging.basicConfig(level=logging.INFO)

# Load bugs
with open('data/GHRB/bug_reports.json') as f:
    bugs = json.load(f)

# Select bug to test
test_bug = bugs[0]  # First bug: assertj-core #2666

print("="*80)
print(f"Testing: {test_bug['project']} #{test_bug['issue_number']}")
print(f"Title: {test_bug['title']}")
print("="*80)

# Initialize pipeline
print("\n1. Initializing pipeline...")
pipeline = LIBROPipeline()

# Load model (use smaller model for testing)
print("\n2. Loading model (5-10 minutes)...")
pipeline.load_model("deepseek-coder-7b")  # Faster than starcoder2-15b
print("✓ Model loaded")

# Configure for quick test
pipeline.config.set("generation.samples_per_bug", 5)  # Just 5 samples for quick test

# Run pipeline
print("\n3. Running pipeline on single bug...")
print("This will take ~3-5 minutes\n")

results = pipeline.run_on_bug(test_bug)

# Show results
print("\n" + "="*80)
print("RESULTS")
print("="*80)

metrics = results['metrics']
print(f"\n📊 Metrics:")
print(f"  Generated tests: {metrics['num_generated']}")
print(f"  Injected tests: {metrics['num_injected']}")
print(f"  FIB tests: {metrics['num_fib']}")
print(f"  BRT tests: {metrics['num_brt']}")
print(f"  Time: {metrics['total_time']:.1f}s")

if metrics['num_brt'] > 0:
    print(f"\n✅ SUCCESS: Found {metrics['num_brt']} BRT(s)!")
    
    print(f"\n📝 First BRT:")
    brt = results['brt_tests'][0]
    print(f"Test code:")
    print("-" * 40)
    print(brt['test_code'])
    print("-" * 40)
else:
    print(f"\n⚠️  No BRT found")
    
    if metrics['num_fib'] > 0:
        print(f"\nFound {metrics['num_fib']} FIB test(s)")
        print("(These fail on buggy version but also fail on fixed version)")
    
    if metrics['num_generated'] > 0 and metrics['num_injected'] == 0:
        print("\n⚠️  Tests were generated but couldn't be injected")
        print("Check logs for details")

# Save results
output_file = Path("results/single_bug_test.json")
output_file.parent.mkdir(parents=True, exist_ok=True)
with open(output_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n💾 Results saved to: {output_file}")